# CIFAR Table 1

TJ Kim

1.17.22

#### Summary:
- Make a table for Benign transferability and inter-boundary distance for following models
- Local benign, fedavg benign, fedEM benign

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [1]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Load blackbox

In [2]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "celeba"
args_.method = "FedAvg"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False
args_.tune_steps = None

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)


np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

args_.save_path = 'weights/blackbox_celeba/'
aggregator.load_state(args_.save_path)

model_weights = []
weights = np.load("weights/blackbox_celeba/train_client_weights.npy")



==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 229.22it/s]


===> Initializing clients..


100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:12<00:00,  6.22it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 3.299 | Train Acc: 2.504% |Test Loss: 3.293 | Test Acc: 2.570% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [3]:
num_models = 1

for i in range(num_models):
    model_weights += [weights[i]]

# Generate the weights to test on as linear combinations of the model_weights
models_test = []

hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

for (w0) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0[0]*weights_h[0][key] 
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]
    
adv_model = copy.deepcopy(models_test[0])
    
del aggregator, clients, weights

### Load Regular

In [12]:
# Load blackbox model
setting = 'local'

if setting == 'FedEM':
    n = 3
else:
    n = 1


# Manually set argument parameters
args_ = Args()
args_.experiment = "celeba"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= n
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/celeba_benign/local/'
args_.validation = False
args_.tune_steps = None

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 228.37it/s]


===> Initializing clients..


100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:09<00:00,  8.50it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 3.299 | Train Acc: 2.504% |Test Loss: 3.293 | Test Acc: 2.570% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [13]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 

Load local model.

In [14]:
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

adv_mode = False
if setting == 'local':

    if adv_mode:
        args_.save_path ='weights/celeba_16_adv/local_defend/'
    else: 
        args_.save_path = 'weights/celeba_benign/local/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
    if adv_mode:
        weights = np.load('weights/celeba_16_adv/local_defend/train_client_weights.npy')
    else:
        weights = np.load("weights/celeba_benign/local/train_client_weights.npy")
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    if adv_mode:
        args_.save_path = 'weights/celeba_16_adv/fed_avg_defend/'
    else:
        args_.save_path = 'weights/celeba_benign/fedavg/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    if adv_mode:
        weights = np.load('weights/celeba_16_adv/fed_avg_defend/train_client_weights.npy')
    else: 
        weights = np.load("weights/celeba_benign/fedavg/train_client_weights.npy")
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    if adv_mode: 
        args_.save_path = 'weights/celeba_16_adv/fedem_defend/'
    else: 
        args_.save_path = 'weights/celeba_benign/fedem/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]
    
    if adv_mode:
        weights = np.load("weights/celeba_16_adv/fedem_defend/train_client_weights.npy")
    else:
        weights = np.load("weights/celeba_benign/fedem/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

Set Up Transfer Attack Scenario

In [15]:
logs_adv = []

for i in range(num_models + 1):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]


In [16]:
all_models = models_test + [adv_model]

In [17]:
# Run Measurements for both targetted and untargeted analysis
victim_idxs = range(num_models)
custom_batch_size = 500
adv_idx = num_models

for vic_idx in victim_idxs:
    print("\t Adv idx:", vic_idx)
    
    dataloader = load_client_data(clients = clients, c_id = vic_idx, mode = 'test') # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    t1 = Transferer(models_list=all_models, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 20,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[vic_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[vic_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[vic_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[vic_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[vic_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack
    t1.atk_params.set_params(batch_size=batch_size, iteration = 20,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 4.5, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[vic_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[vic_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7
	 Adv idx: 8
	 Adv idx: 9
	 Adv idx: 10
	 Adv idx: 11
	 Adv idx: 12
	 Adv idx: 13
	 Adv idx: 14
	 Adv idx: 15
	 Adv idx: 16
	 Adv idx: 17
	 Adv idx: 18
	 Adv idx: 19
	 Adv idx: 20
	 Adv idx: 21
	 Adv idx: 22
	 Adv idx: 23
	 Adv idx: 24
	 Adv idx: 25
	 Adv idx: 26
	 Adv idx: 27
	 Adv idx: 28
	 Adv idx: 29
	 Adv idx: 30
	 Adv idx: 31
	 Adv idx: 32
	 Adv idx: 33
	 Adv idx: 34
	 Adv idx: 35
	 Adv idx: 36
	 Adv idx: 37
	 Adv idx: 38
	 Adv idx: 39


Print Relevant Information

In [18]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([num_models, num_models]) 
orig_sim = np.zeros([num_models, num_models]) 
adv_acc = np.zeros([num_models, num_models]) 
adv_sim_target = np.zeros([num_models, num_models]) 
adv_sim_untarget = np.zeros([num_models, num_models]) 
adv_target = np.zeros([num_models, num_models])
adv_miss = np.zeros([num_models, num_models]) 

for adv_idx in range(num_models):
    for victim in range(num_models):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [19]:
print('adv_target:', avg_nondiag(adv_target), 'std:', np.std(adv_target))
print('adv_miss:', avg_nondiag(adv_miss), 'std:', np.std(adv_miss))
print('orig_acc:', np.mean(np.diagonal(orig_acc)), 'std:', np.std(orig_acc))

adv_target: 0.00041104848601091176 std: 0.0033512896189038348
adv_miss: 0.33564722969328076 std: 0.20902576719736413
orig_acc: 0.6161048961803317 std: 0.24287699160471696


#### Logs